In [ ]:
# Install Libraries
!pip install textblob
!pip install tweepy

In [ ]:
!pip install pycountry
!pip install wordcloud

In [ ]:
# Import Libraries

from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
!pip install langdetect

In [ ]:
# Authentication
consumerKey = "hJmggDw6NDQHKwG8jPyOXYOTF"
consumerSecret = "Cbenb7TZ1roUgOMQJXZyEAfa6uSCexZCs11OQK6SudHzi85Nlr"
accessToken = "1119118220-d8eImP8yfDiC14wUydvDKS8uff9YjM2k28by0wl"
accessTokenSecret = "fxz6DqN2RQorPtVFsKcAFST22ICbZ2IC0z8j6zJFwG5aX"

auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [50]:
def retrive_tweets(api, keywords:[], country_code, from_data, to_data):
    print(f"retrive data in {country_code} at {from_data}")
    keywordStr = "("+" OR ".join(keywords) + ")"
    query = keywordStr + " place_country:" + country_code + " lang:en"
    db = pd.DataFrame(columns=['created_at',
                               'text'])
    for tweet in tweepy.Cursor(api.search_full_archive,
                               query=query,
                               label = 'dev',
                               fromDate=from_data,
                               toDate=to_data).items(200):
        #print(tweet.text)
        ith_tweet = [tweet.created_at, tweet.text]
        db.loc[len(db)] = ith_tweet
    directory = os.path.join(".",country_code)
    if not os.path.exists(directory):
        os.makedirs(directory)
    db.to_csv(os.path.join(directory,country_code+"_"+str(from_data)+".csv"))

In [ ]:
res = retrive_tweets(api, ["Ukraine war","Ukraine conflict"],"US", 202203010000,202203020000)


In [56]:
import datetime
from datetime import timedelta
countries = ["US", "GB", "AU"] # America, United Kingdom, Australia
Keywords = ["Ukraine war","Ukraine conflict"]


startDate = datetime.datetime(2022, 2, 28)
endDate = datetime.datetime(2022, 3, 1)
print(startDate.strftime("%Y%m%d0000"))
currentDate = startDate
data = int(currentDate.strftime("%Y%m%d0000"))
while data < int(endDate.strftime("%Y%m%d0000")):
    nextDate = currentDate+ timedelta(days=1)
    nextDate_int = int(nextDate.strftime("%Y%m%d0000"))
    for country in countries:
        retrive_tweets(api, Keywords,country,data,nextDate_int)
    currentDate = nextDate
    data = nextDate_int

202202280000
retrive data in US at 202202280000


TooManyRequests: 429 Too Many Requests
Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.

In [58]:
# using API v2.0

import requests
import os
import json

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = accessToken

search_url = "https://api.twitter.com/2/tweets/search/recent"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def main():
    json_response = connect_to_endpoint(search_url, query_params)
    print(json.dumps(json_response, indent=4, sort_keys=True))


if __name__ == "__main__":
    main()

401


Exception: (401, '{\n  "title": "Unauthorized",\n  "type": "about:blank",\n  "status": 401,\n  "detail": "Unauthorized"\n}')